In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
import scipy.fftpack
%matplotlib inline

In [ ]:
from analysis.lib.nv import detection
folder= tb.latest_data()
# folder = "D:/measuring/data/20170308/105316_PQ_single_click_ent_111no2_SIL2_phase_stab"
a = m2.M2Analysis(folder)

In [ ]:
# stitch the different part and files together, make and save a plot of the data.
s = []
u_1 = []
v_1 = []
u_2 = []
v_2 = []
w = []

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
sample_counts_1 = a.g['adwindata']['sampling_counts_1'].value
pid_counts_2 = a.g['adwindata']['pid_counts_2'].value
sample_counts_2 = a.g['adwindata']['sampling_counts_2'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = (a.g.attrs['count_int_cycles']/(a.g.attrs['cycle_duration']*100))
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']

# stitch
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
        u_1.append(pid_counts_1[j+i*pid_cycles])
        v_1.append(None)
        u_2.append(pid_counts_2[j+i*pid_cycles])
        v_2.append(None)
        w.append(pid_counts_1[j+i*pid_cycles]*1.04/(pid_counts_1[j+i*pid_cycles]*1.04+pid_counts_2[j+i*pid_cycles]*1.0))
    for k in xrange(sample_cycles):
        u_1.append(None)
        v_1.append(sample_counts_1[k+i*sample_cycles])
        u_2.append(None)
        v_2.append(sample_counts_2[k+i*sample_cycles])
        w.append(sample_counts_1[k+i*sample_cycles]*1.04/(sample_counts_1[k+i*sample_cycles]*1.04+sample_counts_2[k+i*sample_cycles]*1.0))

t = np.arange(0, (len(v_1)*delay/10), (0.1*delay))


fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t, s, 'y')
plt.plot(t, u_1, 'r')
plt.plot(t, v_1, 'b')
plt.title('Counts ZPL detector 1 {0}'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')
ax2 = plt.subplot(212)
plt.plot(t, u_2, 'r')
plt.plot(t, v_2, 'b')
plt.title('Counts ZPL detector 2 {0}'.format(folder))
ax2.set_xlabel('elapsed time (milliseconds)')
ax2.set_ylabel('counts')
# ax3 = plt.subplot(313)
# plt.plot(t, w, 'r')
# plt.title('error signal {0}'.format(folder))
# ax3.set_xlabel('elapsed time (milliseconds)')
# ax3.set_ylabel('error signal')
plt.tight_layout()
# plt.show()

fig.savefig(os.path.join(folder, 'trace_counts.png'))

In [ ]:
zoom_1 = 0*(pid_cycles+sample_cycles)
zoom_2 = 10*(pid_cycles+sample_cycles)

fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t[0:zoom], s[0:zoom], 'y')
plt.plot(t[zoom_1:zoom_2], u_1[zoom_1:zoom_2], 'r')
plt.plot(t[zoom_1:zoom_2], v_1[zoom_1:zoom_2], 'b')

plt.title('Counts ZPL detector 1'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')

ax2 = plt.subplot(212)
plt.plot(t[zoom_1:zoom_2], u_2[zoom_1:zoom_2], 'r')
plt.plot(t[zoom_1:zoom_2], v_2[zoom_1:zoom_2], 'b')

plt.title('Counts ZPL detector 2'.format(folder))
ax2.set_xlabel('elapsed time (milliseconds)')
ax2.set_ylabel('counts')
plt.tight_layout()

fig.savefig(os.path.join(folder, 'trace_counts_zoomed.png'))

In [ ]:
# stitch the different part and files together, make and save a plot of the data.
s = []
u_1 = []

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
sample_counts_1 = a.g['adwindata']['sampling_counts_1'].value
pid_counts_2 = a.g['adwindata']['pid_counts_2'].value
sample_counts_2 = a.g['adwindata']['sampling_counts_2'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = (a.g.attrs['count_int_cycles']/(a.g.attrs['cycle_duration']*100))
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']

# stitch
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
        angle1 = 2*np.arccos((float(pid_counts_1[j+i*pid_cycles])*1.04/(float(pid_counts_1[j+i*pid_cycles]*1.04) + float(pid_counts_2[j+i*pid_cycles])))**(0.5))
        u_1.append(180.0*angle1/(np.pi))
        
       
    for k in xrange(sample_cycles):
        angle2 = 2*np.arccos((float(sample_counts_1[k+i*sample_cycles])*1.04/(float(sample_counts_1[k+i*sample_cycles])*1.04 + float(sample_counts_2[k+i*sample_cycles])))**(0.5))
        u_1.append(180.0*angle2/(np.pi))

t = np.arange(0, (len(v_1)*delay/10), (0.1*delay))


fig = plt.figure(figsize=(17,6))
ax = plt.subplot(111)
plt.plot(t, u_1, 'r')
plt.title('Phase of ZPL photons {0}'.format(folder))
plt.ylim([0,180])
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('angle ($^o$)')


fig.savefig(os.path.join(folder, 'trace_angle.png'))

In [ ]:
zoom_1 = 0*(pid_cycles+sample_cycles)
zoom_2 = 10*(pid_cycles+sample_cycles)

fig = plt.figure(figsize=(17,6))
ax = plt.subplot(111)
plt.plot(t[zoom_1:zoom_2], u_1[zoom_1:zoom_2], 'r')

plt.title('Phase of ZPL photons {0}'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('angle ($^o$)')

fig.savefig(os.path.join(folder, 'trace_angle_zoomed.png'))

In [ ]:
# fourier transform of the sample part

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
sample_counts_1 = a.g['adwindata']['sampling_counts_1'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = (a.g.attrs['count_int_cycles']/(a.g.attrs['cycle_duration']*100))
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']

# stitch
v_1 = []
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
         v_1.append(pid_counts_1[j+i*pid_cycles])
    for k in xrange(sample_cycles):
         v_1.append(sample_counts_1[k+i*sample_cycles])
yf = scipy.fftpack.fft(v_1)

# sample spacing
xf = np.linspace(0, 1.0/(2.0*delay*10**-4), (sample_cycles+pid_cycles)*max_repetitions/2)

index_max = [i for i,v in enumerate(yf) if v > 20000]
frq_max = [x * (1.0/(delay*10**-4)) / ((sample_cycles+pid_cycles)*max_repetitions) for x in index_max]
print 'peak frequencies are:'
print frq_max

norm1 = yf / np.linalg.norm(yf)

fig, ax = plt.subplots()
plt.ylim([0,20000])
ax.plot(xf, yf[:len(yf)/2])
plt.title('FFT {0}'.format(folder))
ax.set_xlabel('frequency (Hz)')
ax.set_ylabel('Amplitude (a.u.)')
# ax.set_yticklabels([])
fig.savefig(os.path.join(folder, 'fft.png'))